In [1]:
MODEL_DIR = None
MASK_CLS = 'ilm.mask.hierarchical.MaskHierarchical'

In [2]:
# Download pretrained model

if MODEL_DIR is None:
    !python acl20_repro.py model sto ilm | bash
    MODEL_DIR = '/tmp/ilm/models/sto_ilm'

--2021-02-28 10:01:35--  https://docs.google.com/uc?export=download&confirm=4oPS&id=1oYFLxkX6mWbmpEwQH8BmgE7iKix2W0zU
Resolving docs.google.com (docs.google.com)...2404:6800:4003:c00::71, 2404:6800:4003:c00::65, 2404:6800:4003:c00::8a, ...
Connecting to docs.google.com (docs.google.com)|2404:6800:4003:c00::71|:443...connected.
HTTP request sent, awaiting response...302 Moved Temporarily
Location: https://doc-0k-4g-docs.googleusercontent.com/docs/securesc/0mjtlf2845dt1a1b2cni9a68p1t7f2gv/bgqsv57qqoub794dehmd4a1v9c3d5edh/1614481275000/00801004230948063963/18148810083193681572Z/1oYFLxkX6mWbmpEwQH8BmgE7iKix2W0zU?e=download [following]
--2021-02-28 10:01:35--  https://doc-0k-4g-docs.googleusercontent.com/docs/securesc/0mjtlf2845dt1a1b2cni9a68p1t7f2gv/bgqsv57qqoub794dehmd4a1v9c3d5edh/1614481275000/00801004230948063963/18148810083193681572Z/1oYFLxkX6mWbmpEwQH8BmgE7iKix2W0zU?e=download
Resolving doc-0k-4g-docs.googleusercontent.com (doc-0k-4g-docs.googleusercontent.com)... 2404:6800:4003:c03::

In [2]:
# Prepare tokenizer

import os
import pickle

import ilm.tokenize_util

tokenizer = ilm.tokenize_util.Tokenizer.GPT2
with open(os.path.join(MODEL_DIR, 'additional_ids_to_tokens.pkl'), 'rb') as f:
    additional_ids_to_tokens = pickle.load(f)
additional_tokens_to_ids = {v:k for k, v in additional_ids_to_tokens.items()}
try:
    ilm.tokenize_util.update_tokenizer(additional_ids_to_tokens, tokenizer)
except ValueError:
    print('Already updated')
print(additional_tokens_to_ids)

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [4]:
# Load model

import torch
from transformers import GPT2LMHeadModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GPT2LMHeadModel.from_pretrained(MODEL_DIR)
model.eval()
_ = model.to(device)

In [5]:
# Create context

context = """
Math Class
Chris was bad at _. _ _ _ He ended up passing the test.
""".strip()

context_ids = ilm.tokenize_util.encode(context, tokenizer)

# Replace blanks with appropriate tokens from left to right
_blank_id = ilm.tokenize_util.encode(' _', tokenizer)[0]
context_ids[context_ids.index(_blank_id)] = additional_tokens_to_ids['<|infill_word|>']
context_ids[context_ids.index(_blank_id)] = additional_tokens_to_ids['<|infill_sentence|>']
context_ids[context_ids.index(_blank_id)] = additional_tokens_to_ids['<|infill_sentence|>']
context_ids[context_ids.index(_blank_id)] = additional_tokens_to_ids['<|infill_sentence|>']
print(ilm.tokenize_util.decode(context_ids, tokenizer))

Math Class
Chris was bad at<|infill_word|>.<|infill_sentence|><|infill_sentence|><|infill_sentence|> He ended up passing the test.


In [11]:
from ilm.infer import infill_with_ilm

generated = infill_with_ilm(
    model,
    additional_tokens_to_ids,
    context_ids,
    num_infills=2)
for g in generated:
    print('-' * 80)
    print(ilm.tokenize_util.decode(g, tokenizer))

--------------------------------------------------------------------------------
Math Class
Chris was bad at math. He needed to do well on the test. He decided to study more. Chris spent all day studying. He ended up passing the test.
--------------------------------------------------------------------------------
Math Class
Chris was bad at math. He decided to go to a class to improve. His teacher learned that Chris was failing math. Chris decided to take his own math class. He ended up passing the test.
